In [26]:
from texascovid19 import constants
import requests
import pandas as pd
from datetime import date

Fetch each day's data.  The data is "current as of 8 p.m. the day before reporting."

In [27]:
r = requests.get(constants.DSHS_XHR_URL, verify=False)
r.raise_for_status()

/Users/dharbin/work/texascovid19/venv/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'services5.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [28]:
county_data = r.json()['features']

In [29]:
norm_county_data = []
for county in county_data:
        norm_county_data.append({
            'County of Residence': county['attributes']['County'],
            'Deaths': county['attributes']['Fatalities'],
            'Number of Cases': county['attributes']['Positive'],
            'Recoveries': county['attributes']['Recoveries'],
            'Active': county['attributes']['Active']
        })

In [30]:
df = pd.DataFrame(data=norm_county_data)

In [31]:
df

,County of Residence,Deaths,Number of Cases,Recoveries,Active
0,Harris,183,8416,3213,4784
1,Dallas,148,6359,2699,3279
2,Tarrant,109,3810,809,2832
3,Travis,71,2235,760,1346
4,Bexar,57,1942,1028,835
...,...,...,...,...,...
214,Knox,0,1,1,0
215,Mitchell,0,1,1,0
216,Motley,0,1,1,0
217,Reeves,0,1,0,1


In [32]:
df.set_index("County of Residence", inplace=True)

In [33]:
today = date.today().isoformat()

In [34]:
df.to_csv(f"{constants.DATA_PATH}/{today}.csv", header=True, index=True)

Append today's data to timeseries

In [35]:
df_timeseries = pd.read_csv(f"{constants.TIMESERIES_CSV}").set_index("County of Residence")

In [36]:
df_copy = df.copy()

In [37]:
df_copy.rename(columns={"Number of Cases": today}, inplace=True)

In [38]:
for col in df_copy.columns:
    if col != today:
        df_copy.drop(col, axis=1, inplace=True)

In [39]:
if today in df_timeseries.columns:
    df_timeseries.drop(columns=today, inplace=True)

In [40]:
df_timeseries = pd.concat([df_timeseries, df_copy], axis=1).rename_axis('County of Residence')

In [41]:
df_timeseries = df_timeseries.fillna(0).convert_dtypes(convert_integer=True)

In [42]:
df_timeseries.to_csv(constants.TIMESERIES_CSV, header=True, index=True)

Also for deaths

In [43]:
df_timeseries = pd.read_csv(f"{constants.DEATHS_TIMESERIES_CSV}").set_index("County of Residence")

In [44]:
df_copy = df.copy()

In [45]:
df_copy.rename(columns={"Deaths": today}, inplace=True)

In [46]:
for col in df_copy.columns:
    if col != today:
        df_copy.drop(col, axis=1, inplace=True)

In [47]:
if today in df_timeseries.columns:
    df_timeseries.drop(columns=today, inplace=True)

In [48]:
df_timeseries = pd.concat([df_timeseries, df_copy], axis=1).rename_axis('County of Residence')

In [49]:
df_timeseries = df_timeseries.fillna(0).convert_dtypes(convert_integer=True)

In [50]:
df_timeseries.to_csv(constants.DEATHS_TIMESERIES_CSV, header=True, index=True)